In [1]:
import pandas as pd

from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm.exc import NoResultFound

from models import Program, Block, Workout, Workout_set, Exercise

In [2]:
engine = create_engine(f"sqlite:///data/db/gym_database.db")
Session = sessionmaker(bind=engine)
session = Session()

# Algorithm

 1. Read excel (when log data is loaded, it's gonna overwrite everything by default)
 2. For each Block (=sheet), iterate through Workouts
     1. Identify Workout (using Program-Block-Workout)
     2. Log Workout data (header excel data for workout)
     3. Iterate through workout table data
         1. Identify no. of sets per Exercise line
         2. Assign correct characterization to each line
         3. Proceed with following exercise  

In [8]:
FILE = "/mnt/c/Users/gonza/OneDrive/Gym/routines_log/test.xlsx"
# FILE = "/mnt/d/OneDrive/Gym/routines_log/test2.xlsx"

for block_name, df_block in pd.read_excel(FILE, sheet_name=None, header=None).items():
    print(block_name)
    # display(df_block)
    df = df_block
    

Micro 0 Gonzalo septiembre2021
Micro1 Gonzalo septiembre2021
Micro2 Gonzalo septiembre2021


In [9]:
idx = df.index[df.isna().all(axis=1)].tolist()
# We add the -1 to then add 1 in the loop and avoid including empty rows
idx_mod = [-1] + idx + [len(df)]

for i in range(len(idx_mod)-1):
    df_wod = df.iloc[idx_mod[i]+1:idx_mod[i+1]]

    df_wod_header = df_wod.loc[df_wod.iloc[:, 2:].isna().all(axis=1)].dropna(axis=1, how="all")
    display(df_wod_header)

    df_wod_exer = df_wod.loc[df_wod.iloc[:, 2:].notna().any(axis=1)]
    df_wod_exer.columns = df_wod_exer.iloc[0]
    df_wod_exer = df_wod_exer.iloc[1:]
    display(df_wod_exer)

,0,1
0,Fecha,2021-09-20 00:00:00
1,Desc,pierna
2,Duración (min),70
3,RPE general,7
4,Comentario general,NaN


5,Ejercicio,Series,Repeticiones,Peso (kg),% 1RM,RPE mín.,RPE máx.,Descanso (min),¿Hecho?,RPE,Comentarios
6,"sentadilla excéntrica 3""",4,7,70,70,NaN,8,3,x,8,NaN
7,"peso muerto excéntrico 3""",4,7,85,65,NaN,8,3,x,8,NaN
8,front squat,5,12,30,70,NaN,8,2,x,7,NaN
9,sentadilla búlgara,3,8,10,70,NaN,7,1,x,7,NaN
10,rueda abd,4,12,2,70,NaN,7,1,x,7,NaN
11,"plancha abd 30""",4,1,2,70,NaN,7,1,x,7,NaN


,0,1
13,Fecha,2021-09-21 00:00:00
14,Desc,torso
15,Duración (min),75
16,RPE general,7
17,Comentario general,NaN


18,Ejercicio,Series,Repeticiones,Peso (kg),% 1RM,RPE mín.,RPE máx.,Descanso (min),¿Hecho?,RPE,Comentarios
19,press de banca con pausa,4,6,75,75,NaN,7,3,x,8,NaN
20,fondos en paralelas (abiertos),4,10,80,70,NaN,8,2,x,7,NaN
21,flexiones de pecho,4,10,40,70,NaN,5,2,x,5,NaN
22,dominadas de biceps lastradas +25kg,4,5,105,85,NaN,8,3,x,6,NaN
23,"dominadas de biceps excéntricas 3""",4,6,80,70,NaN,7,2,x,7,NaN
24,extension de tríceps en polea con cuerda,6,15,20,70,NaN,8,1,x,8,NaN


,0,1
26,Fecha,2021-09-23 00:00:00
27,Desc,acc piern pst
28,Duración (min),80
29,RPE general,8
30,Comentario general,NaN


31,Ejercicio,Series,Repeticiones,Peso (kg),% 1RM,RPE mín.,RPE máx.,Descanso (min),¿Hecho?,RPE,Comentarios
32,peso muerto sumo,4,10,85,85,NaN,8,3,x,6,NaN
33,hip thrust,3,10,120,85,NaN,8,2,x,8,NaN
34,hip thrust,3,12,100,75,NaN,8,2,x,8,NaN
35,hip thrust,3,15,85,65,NaN,10,2,x,10,NaN
36,femoral en maquina,5,12,30,75,NaN,7,1,x,8,NaN
37,femoral con mancuernas,5,12,15,70,NaN,8,1,x,6,NaN


,0,1
39,Fecha,2021-09-25 00:00:00
40,Desc,acc torso
41,Duración (min),65
42,RPE general,8
43,Comentario general,NaN


44,Ejercicio,Series,Repeticiones,Peso (kg),% 1RM,RPE mín.,RPE máx.,Descanso (min),¿Hecho?,RPE,Comentarios
45,press inclinado,"2, 2",10,"60, 50",60,NaN,7,2,x,8,Muy forzado (2 últimas series con 50kg) porque...
46,apertura de mancuernas en banco inclinado,4,10,12,70,NaN,8,1,x,8,Muy forzado porque empecé por Press Militar
47,remo barra,4,10,60,60,NaN,8,2,x,7,NaN
48,remo con mancuernas en banco inclinado,4,12,20,80,NaN,8,1,x,9,NaN
49,press militar,4,10,40,80,NaN,8,2,x,8,NaN
50,press militar,4,15,30,70,NaN,9,2,x,9,Alguna molestia en hombro izquierdo cuando est...
51,face pull,4,12,30,80,NaN,8,1,x,8,NaN
52,face pull,4,15,20,70,NaN,8,1,x,8,NaN


In [10]:
df_wod_header = df_wod.loc[df_wod.iloc[:, 2:].isna().all(axis=1)].dropna(axis=1, how="all")
df_wod_header
df_wod_exer = df_wod.loc[df_wod.iloc[:, 2:].notna().any(axis=1)]
df_wod_exer.columns = df_wod_exer.iloc[0]
df_wod_exer = df_wod_exer.iloc[1:]
df_wod_exer

44,Ejercicio,Series,Repeticiones,Peso (kg),% 1RM,RPE mín.,RPE máx.,Descanso (min),¿Hecho?,RPE,Comentarios
45,press inclinado,"2, 2",10,"60, 50",60,NaN,7,2,x,8,Muy forzado (2 últimas series con 50kg) porque...
46,apertura de mancuernas en banco inclinado,4,10,12,70,NaN,8,1,x,8,Muy forzado porque empecé por Press Militar
47,remo barra,4,10,60,60,NaN,8,2,x,7,NaN
48,remo con mancuernas en banco inclinado,4,12,20,80,NaN,8,1,x,9,NaN
49,press militar,4,10,40,80,NaN,8,2,x,8,NaN
50,press militar,4,15,30,70,NaN,9,2,x,9,Alguna molestia en hombro izquierdo cuando est...
51,face pull,4,12,30,80,NaN,8,1,x,8,NaN
52,face pull,4,15,20,70,NaN,8,1,x,8,NaN


In [11]:
import re

txt = "2, 2, 5.1"

# [int(s) for s in txt.split() if s.isdigit()]
# [int(s) for s in re.findall(r"\d+", txt)]
[int(s) for s in re.split(r", |,", txt) if s.isnumeric()]

txt = "60, 50"
[float(s) for s in re.split(r", |,", txt)]

[60.0, 50.0]

In [12]:
df = df_wod_exer.astype({"Series": int, 
                         "Repeticiones": int,
                         "Peso (kg)": float,
                         "% 1RM": float,
                         "RPE mín.": int,
                         "RPE máx.": int, 
                         "Descanso (min)": float,
                         "RPE": int}, 
                        errors="ignore")

for _, row in df.iterrows():
    if isinstance(row["Series"], int):
        # print(row)
        for set_id in range(1, row["Series"]+1):
            print(set_id)
            print(row.drop("Series"))
    else:
        no_sets = [int(s) for s in re.split(r", |,", row["Series"]) if s.isnumeric()]
        try:
            weights = [float(s) for s in re.split(r", |,", row["Peso (kg)"])]
        except ValueError:
            raise("Not a float!")
        
        set_id = 0
        for idx, sets in enumerate(no_sets):
            for set_ in range(1, sets+1):
                set_id += 1
                print(set_id)
                print(row.drop(["Series", "Peso (kg)"]))
                print(weights[idx])

# type(df["Series"].iloc[1])

1
44
Ejercicio                                           press inclinado
Repeticiones                                                     10
% 1RM                                                          60.0
RPE mín.                                                        NaN
RPE máx.                                                          7
Descanso (min)                                                  2.0
¿Hecho?                                                           x
RPE                                                               8
Comentarios       Muy forzado (2 últimas series con 50kg) porque...
Name: 45, dtype: object
60.0
2
44
Ejercicio                                           press inclinado
Repeticiones                                                     10
% 1RM                                                          60.0
RPE mín.                                                        NaN
RPE máx.                                                          7
Descanso 

In [13]:
pd.read_sql(
    session.query(Workout_set.workout_set_id,
                  Exercise.exercise_desc.label("Ejercicio"),
                  Workout_set.set_id.label("Serie ID"),
                  Workout_set.no_reps.label("Repeticiones"),
                  Workout_set.weight.label("Peso (kg)"),
                  Workout_set.perc_rm.label("% 1RM"),
                  Workout_set.min_rpe.label("RPE mín."),
                  Workout_set.max_rpe.label("RPE máx."),
                  Workout_set.rest_min.label("Descanso (min)"))
    # .group_by(Exercise.exercise_desc)
    .filter(Exercise.exercise_id == Workout_set.exercise_id,
            Workout_set.workout_id == 15)
    .order_by(Workout_set.workout_set_id)
    .statement,
session.bind)

,workout_set_id,Ejercicio,Serie ID,Repeticiones,Peso (kg),% 1RM,RPE mín.,RPE máx.,Descanso (min)
0,269,press inclinado,1,10,60.0,60.0,None,7,2.0
1,270,press inclinado,2,10,60.0,60.0,None,7,2.0
2,271,press inclinado,3,10,60.0,60.0,None,7,2.0
3,272,press inclinado,4,10,60.0,60.0,None,7,2.0
4,273,apertura de mancuernas en banco inclinado,1,10,12.0,70.0,None,8,1.0
5,274,apertura de mancuernas en banco inclinado,2,10,12.0,70.0,None,8,1.0
6,275,apertura de mancuernas en banco inclinado,3,10,12.0,70.0,None,8,1.0
7,276,apertura de mancuernas en banco inclinado,4,10,12.0,70.0,None,8,1.0
8,277,remo barra,1,10,60.0,60.0,None,8,2.0
9,278,remo barra,2,10,60.0,60.0,None,8,2.0


In [ ]:
pd.read_sql(
    session.query(Workout)
)